In [1]:
import requests
import pandas as pd

import datetime
import reverse_geocoder as rg

https://public.emdat.be/data <br>
datos sobre fallecidos por año en todos los países

# Extract

In [2]:
df = pd.read_excel('../../DataSet/raw/emdat_public_2023_07_05_query_uid-U68zMR.xlsx',skiprows=6)
df

c:\Users\sgraf\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1906-0002-COL,1906,2,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077011,NaN,NaN,NaN,NaN
1,1950-0028-COL,1950,28,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,5000.0,60683.0,8.239552,NaN,NaN,NaN,NaN
2,1962-0030-COL,1962,30,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.336410,NaN,NaN,NaN,NaN
3,1967-0020-COL,1967,20,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,600.0,5264.0,11.398527,NaN,NaN,NaN,NaN
4,1970-0051-COL,1970,51,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,400.0,3015.0,13.266480,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,1973-0148-USA,1973,148,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,1000.0,6591.0,15.171453,NaN,NaN,NaN,NaN
142,1973-0149-USA,1973,149,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,5750.0,37900.0,15.171453,NaN,NaN,NaN,NaN
143,1975-0155-USA,1975,155,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,1000.0,5438.0,18.389129,NaN,NaN,NaN,NaN
144,2022-0808-USA,2022,808,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,250000.0,250000.0,100.000000,2,NaN,28876,Humboldt (Adm2).


# Transform

In [3]:
df.columns

Index(['Dis No', 'Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'AID Contribution ('000 US$)', 'Dis Mag Value',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')

dejamos las columnas que nos interesan

In [4]:
df = df[['Year', 'Disaster Subtype',
'ISO', 'Location',
'Origin',
'Dis Mag Value',
'Latitude', 'Longitude', 'Local Time',
'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 
'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
'Total Affected',
"Insured Damages ('000 US$)", "Insured Damages, Adjusted ('000 US$)",
"Total Damages ('000 US$)", "Total Damages, Adjusted ('000 US$)"]].copy()

In [5]:
df.rename(columns={'No Injured':'Injured','No Affected':'Affected', 'No Homeless':'Homeless'}, inplace=True)
df.rename(columns={"Insured Damages ('000 US$)":"Insured Damages US$", "Insured Damages, Adjusted ('000 US$)": "Insured Damages US$(2021)",
                   "Total Damages ('000 US$)":"Total Damages US$", "Total Damages, Adjusted ('000 US$)": "Total Damages US$(2021)"}, inplace=True)

Juntar columnas de fecha

In [6]:
df['Start Year'] = df['Start Year'].astype(str)
df['Start Month'] = df['Start Month'].astype(str)
df['Start Day'] = df['Start Day'].astype(str)
df['Start Year'] = df[['Start Year', 'Start Month', 'Start Day']].agg('-'.join, axis=1).apply(lambda x: pd.to_datetime(x, format='%Y/%m/%d'))
df.rename(columns={'Start Year':'date'}, inplace=True)
df.drop(columns=['Start Day', 'Start Month'],inplace=True)

df['End Year'] = df['End Year'].astype(str)
df['End Month'] = df['End Month'].astype(str)
df['End Day'] = df['End Day'].astype(str)
df['End Year'] = df[['End Year', 'End Month', 'End Day']].agg('-'.join, axis=1).apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
df.rename(columns={'End Year':'End date'},inplace=True)
df.drop(columns=['End Day', 'End Month'],inplace=True)

In [7]:
df['Total Damages US$']

0           NaN
1        5000.0
2           NaN
3         600.0
4         400.0
         ...   
141      1000.0
142      5750.0
143      1000.0
144    250000.0
145         NaN
Name: Total Damages US$, Length: 146, dtype: float64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Year                       146 non-null    int64         
 1   Disaster Subtype           146 non-null    object        
 2   ISO                        146 non-null    object        
 3   Location                   144 non-null    object        
 4   Origin                     5 non-null      object        
 5   Dis Mag Value              141 non-null    float64       
 6   Latitude                   141 non-null    object        
 7   Longitude                  141 non-null    object        
 8   Local Time                 111 non-null    object        
 9   date                       146 non-null    datetime64[ns]
 10  End date                   146 non-null    datetime64[ns]
 11  Total Deaths               106 non-null    float64       
 12  Injured 

In [9]:
df.notnull().sum().sort_values(ascending=False) / df.notnull().sum().sort_values(ascending=False)[0] * 100

Year                         100.000000
date                         100.000000
Disaster Subtype             100.000000
End date                     100.000000
ISO                          100.000000
Location                      98.630137
Dis Mag Value                 96.575342
Latitude                      96.575342
Longitude                     96.575342
Total Affected                82.191781
Local Time                    76.027397
Total Deaths                  72.602740
Injured                       68.493151
Total Damages US$             62.328767
Total Damages US$(2021)       62.328767
Affected                      47.260274
Homeless                      27.397260
Insured Damages US$           21.917808
Insured Damages US$(2021)     21.917808
Origin                         3.424658
dtype: float64

In [10]:
df.drop(columns=['Origin','Injured','Affected','Homeless', 'Insured Damages US$', 'Insured Damages US$(2021)', 'Total Damages US$'],inplace=True)

### Sacar las coordenadas desde 'Location'

In [29]:
from geopy.geocoders import Nominatim

def obtener_coordenadas_localidad(localidad):
    try:
        geolocator = Nominatim(user_agent="rafaelnsegurag@gmail.com")  # Puedes proporcionar un nombre de usuario único para el parámetro 'user_agent'
        location = geolocator.geocode(localidad)
        if location:
            return location.latitude, location.longitude
        else:
            return None
    except:
        return None

In [30]:
noL = df[df['Latitude'].isna()]
noL['Location'].apply(lambda x: pd.Series(obtener_coordenadas_localidad(x) if x else [None, None], dtype='object'))

""
27
29
30
118
145


In [30]:
noL['Location']

27                                         Hawai
29                        Onagawa, Honshu Island
30                                        Hawaii
118    Atsuma village, Abira, Sapporo (Hokkaido)
145                                     Ishikawa
Name: Location, dtype: object

In [20]:
noL

,Year,Disaster Subtype,ISO,Location,Dis Mag Value,Latitude,Longitude,Local Time,date,End date,Total Deaths,Total Affected,Total Damages US$(2021)
27,1952,Tsunami,USA,Hawai,NaN,NaN,NaN,NaN,1952-11-04,1952-11-04,NaN,NaN,9896.0
29,1960,Tsunami,JPN,"Onagawa, Honshu Island",NaN,NaN,NaN,NaN,1960-05-22,1960-05-22,138.0,NaN,1385348.0
30,1960,Tsunami,USA,Hawaii,10.0,NaN,NaN,NaN,1960-05-23,1960-05-23,61.0,NaN,5689823.0
118,2018,Ground movement,JPN,"Atsuma village, Abira, Sapporo (Hokkaido)",7.0,NaN,NaN,03:00,2018-09-06,2018-09-06,41.0,6280.0,1456825.0
145,2023,Ground movement,JPN,Ishikawa,6.0,NaN,NaN,14:42,2023-05-06,2023-05-06,1.0,1084.0,NaN


# Load

In [11]:
df.to_csv('../../DataSet/processed/emdat_CO_EU_JP_2023_07_05.csv', index=False)